In [1]:
import numpy as np
from steps import *
from tqdm import tqdm
from optimal import *
import matplotlib.pyplot as plt
import optuna
import pickle
import warnings
warnings.filterwarnings("error")


In [2]:
def plot_res(x_axis, results, x_label, y_label, label, title, path):
    plt.grid()
    plt.plot(x_axis, results.mean(axis=1), label=label)
    plt.fill_between(x_axis, results.mean(axis=1)+results.std(axis=1), results.mean(axis=1)-results.std(axis=1),
                     color='r', alpha=0.5)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.savefig(path+".png")
    plt.clf()

    plt.grid()
    plt.boxplot(results.T, positions=x_axis)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.savefig("BP-"+path+".png")
    plt.clf()

In [6]:
d = 20
opt_r = 5
m = 20
T_r = 100
times = 10
opt_sigma = 3
sigma_type = matrix_type.DIAG
s_type = matrix_type.IDENTITY

name = f"optuna_optimization_d={d}_r={opt_r}_sigma={sigma_type}_s={s_type}_m={m}_Tr={T_r}_var={opt_sigma}"
study = optuna.create_study(study_name=f'{name}',
                            storage=f'sqlite:///./../optuna/{name}.db ',
                            direction='minimize', load_if_exists=True)

[I 2023-05-14 22:02:28,060] Using an existing study with name 'optuna_optimization_d=20_r=5_sigma=matrix_type.DIAG_s=matrix_type.IDENTITY_m=20_Tr=100_var=3' instead of creating a new one.


In [7]:
# Sigma
sigma_max = 10.0
sigma_min = 0.5
spaces = 0.5
load = False

suffix = f'{sigma_min}-{sigma_max}-{spaces}_opt-r={opt_r}_opt-sigma={opt_sigma}_NEW'

if not load:
    d_sigmas = np.linspace(sigma_min, sigma_max, int((sigma_max-sigma_min)/spaces)+1)
    results = np.zeros([len(d_sigmas), times])
    for idx, d_sigma in enumerate(d_sigmas):
        print(f" ***** Sigma={d_sigma} *****")
        regret_list, regret_mix_list = run_algorithm(d=d, r=opt_r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=d_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_sigma0_{suffix}.p', 'wb'))
    pickle.dump(d_sigmas, open(f'sigma0-acc_{suffix}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_sigma0_{suffix}.p', 'rb'))
    d_sigmas = pickle.load(open(f'sigma0-acc_{suffix}.p', 'rb'))

plot_res(x_axis=d_sigmas, results=results,
         x_label=r'$\sigma_0$', y_label='Accuracy Ratio', label=f'd={d}, r={opt_r}',
         title=r'results accuracy ratio vs $\sigma_0$',
         path=f"results_accuracy_ratio_sigma0_{suffix}")

 ***** Sigma=0.5 *****
l_star=15 ; regret_mix=0.7486915554888657 ; last_regret=0.862786625492536
l_star=16 ; regret_mix=0.6833666686333392 ; last_regret=0.797616063980781
l_star=15 ; regret_mix=0.719099978806231 ; last_regret=0.849659029352324
l_star=14 ; regret_mix=0.697239200338696 ; last_regret=0.8161104501365416
l_star=10 ; regret_mix=0.6462506483551432 ; last_regret=0.7989700479293624
l_star=14 ; regret_mix=0.7083146368824392 ; last_regret=0.8327740940741558
l_star=14 ; regret_mix=0.6683492687728976 ; last_regret=0.8216273880654689
l_star=15 ; regret_mix=0.7219688320015422 ; last_regret=0.8379122802982878
l_star=12 ; regret_mix=0.7009535426776482 ; last_regret=0.8489773465199765
l_star=14 ; regret_mix=0.6912964914158197 ; last_regret=0.8244299685882795
 ***** Sigma=1.0 *****
l_star=10 ; regret_mix=0.5110655699571635 ; last_regret=0.6105850875853988
l_star=10 ; regret_mix=0.5307772374124031 ; last_regret=0.6415298383663028
l_star=10 ; regret_mix=0.44708027233105363 ; last_regret=0.

KeyboardInterrupt: 

In [12]:
# r
r_max = 10
r_min = 1
spaces = 1
load = False

suffix = f'{r_min}-{r_max}-{spaces}_opt-r={opt_r}_opt-sigma={opt_sigma}'

if not load:
    r_list = range(r_min, r_max, spaces)
    results = np.zeros([len(r_list), times])
    for idx, r in enumerate(r_list):
        regret_list, regret_mix_list = run_algorithm(d=d, r=r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=opt_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_r_.p', 'wb'))
    pickle.dump(r_list, open(f'r-acc_{suffix}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_r_{suffix}.p', 'rb'))
    r_list = pickle.load(open(f'r-acc_{suffix}.p', 'rb'))

plot_res(x_axis=r_list, results=results,
         x_label='r', y_label='Accuracy Ratio', label=f"d={d}"+r" $\sigma_0$="+f"{opt_sigma}",
         title='results accuracy ratio vs r',
         path=f"results_accuracy_ratio_r_{suffix}")


l_star=2 ; regret_mix=0.9070362437226382 ; last_regret=0.9400492446699711
l_star=2 ; regret_mix=2.7696551210299725 ; last_regret=2.8209148396769614
l_star=2 ; regret_mix=3.2955605046257817 ; last_regret=3.3487057891737533
l_star=2 ; regret_mix=4.023268112306739 ; last_regret=4.086966903379754
l_star=2 ; regret_mix=0.050736776024568164 ; last_regret=0.05055405320384303
l_star=2 ; regret_mix=0.6670139763774006 ; last_regret=0.6802631296936584
l_star=2 ; regret_mix=0.10691647575756723 ; last_regret=0.1074039457909633
l_star=2 ; regret_mix=2.7868519198413546 ; last_regret=2.94194504660102
l_star=2 ; regret_mix=4.896108111195703 ; last_regret=4.998226623210869
l_star=2 ; regret_mix=4.0365017848736295 ; last_regret=4.073216146184629
l_star=3 ; regret_mix=0.26923749527947866 ; last_regret=0.28481492553252935
l_star=3 ; regret_mix=0.31659829004530216 ; last_regret=0.3437736228500176
l_star=3 ; regret_mix=1.1546644765938663 ; last_regret=1.3156574174568247
l_star=3 ; regret_mix=0.02426091576685

Exception ignored in: <_io.FileIO name='acc-ratio_vs_r_.p' mode='wb' closefd=True>
Exception ignored in: <_io.FileIO name='r-acc_1-10-1_opt-r=3_opt-sigma=5.p' mode='wb' closefd=True>
No handles with labels found to put in legend.


last_regret=0.0005948031618093453


<Figure size 432x288 with 0 Axes>

In [4]:
# d
load = False
d_max = 20
d_min = opt_r+2
spaces = 1
suffix = f'{d_min}-{d_max}-{spaces}_opt-r={opt_r}_opt-sigma={opt_sigma}'

if not load:
    d_list = range(d_min, d_max, spaces)
    results = np.zeros([len(d_list), times])

    for idx, d in enumerate(d_list):
        regret_list, regret_mix_list = run_algorithm(d=d, r=opt_r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=opt_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_d_{suffix}.p', 'wb'))
    pickle.dump(d_list, open(f'd-acc_{suffix}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_d_{suffix}.p', 'rb'))
    d_list = pickle.load(open(f'd-acc_{suffix}.p', 'rb'))

plot_res(x_axis=d_list, results=results,
         x_label='d', y_label='Accuracy Ratio', label=f"r={opt_r}"+r" $\sigma_0$="+f"{opt_sigma}",
         title='results accuracy ratio vs d',
         path=f"results_accuracy_ratio_d_{suffix}")

l_star=4 ; regret_mix=0.0077437513816775 ; last_regret=0.0014317817711535584
l_star=4 ; regret_mix=0.09818390473117763 ; last_regret=0.10970264137724482
l_star=4 ; regret_mix=0.00046250233172237775 ; last_regret=0.00041814509647950123
l_star=4 ; regret_mix=0.001068100199516901 ; last_regret=0.0006050303654470937
l_star=4 ; regret_mix=2.8021471079706566e-05 ; last_regret=2.052990036614262e-05
l_star=4 ; regret_mix=0.024594978354402622 ; last_regret=0.026547678962056665
l_star=4 ; regret_mix=5.372428272721262e-05 ; last_regret=1.6029484575241781e-06
l_star=4 ; regret_mix=6.226457153275165e-07 ; last_regret=1.1815603125836968e-06
l_star=4 ; regret_mix=1.1399355593317228e-05 ; last_regret=1.4262393356711185e-05
l_star=4 ; regret_mix=0.1049034226933166 ; last_regret=0.11309066207505176
l_star=4 ; regret_mix=0.0051234912444659084 ; last_regret=0.0035267117407893358
l_star=4 ; regret_mix=0.027020275790337923 ; last_regret=0.029873128052318385
l_star=4 ; regret_mix=0.1373985271021085 ; last_re

Exception ignored in: <_io.FileIO name='acc-ratio_vs_d_5-20-1_opt-r=3_opt-sigma=5.p' mode='wb' closefd=True>


NameError: name 'r_list' is not defined

In [6]:
# Regrets
load = False

if not load:
    d=20
    regret_list, regret_mix_list = run_algorithm(d=d, r=opt_r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=opt_sigma, T_r=T_r,
                                                     **study.best_params)
    pickle.dump(regret_list, open(f'regrets.p', 'wb'))

else:
    regret_list = pickle.load(open(f'regrets.p', 'rb'))


plot_res(x_axis=range(m+2), results=regret_list.T,
         x_label='#Iteration', y_label='Regret', label=f'd={d} r={opt_r} '+r'$\sigma_0$='+f'{opt_sigma}',
         title='Regret vs #Iteration',
         path=f"Regret_vs_Iteration_r={opt_r}_var={opt_sigma}")


l_star=4 ; regret_mix=0.48280532784382285 ; last_regret=0.6728611079598498
l_star=4 ; regret_mix=0.007746272830328938 ; last_regret=0.006619803264309738
l_star=5 ; regret_mix=0.16938401460103533 ; last_regret=0.18874977236761356
l_star=4 ; regret_mix=0.013396885382376269 ; last_regret=0.013462988407887158
l_star=4 ; regret_mix=0.09970592419188108 ; last_regret=0.1223157128427819
l_star=5 ; regret_mix=0.12935182298646677 ; last_regret=0.15413115569755975
l_star=4 ; regret_mix=0.2889069403201405 ; last_regret=0.32546988535779864
l_star=4 ; regret_mix=0.31223378359854936 ; last_regret=0.3596080453044863
l_star=4 ; regret_mix=0.6357745281474471 ; last_regret=0.9758236389861856
l_star=4 ; regret_mix=0.7561700956234414 ; 

Exception ignored in: <_io.FileIO name='regrets.p' mode='wb' closefd=True>
No handles with labels found to put in legend.


last_regret=1.055171859164946


<Figure size 432x288 with 0 Axes>